In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
%cd ..

/home/den/dev/git/ozon-e-cup-2025


In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline

import src.preprocessing as prep

In [ ]:
train = pd.read_csv("data/1__train_with_flags.csv")
test = pd.read_csv("data/1__test_with_flags.csv")

# Numeric features preprocessing

In [ ]:
num_prep_pipe = prep.NUMERIC_PREP_PIPELINE
num_prep_pipe = num_prep_pipe.fit(train)

[Pipeline] ... (step 1 of 37) Processing rating_1_count, total=   0.1s
[Pipeline] ... (step 2 of 37) Processing rating_2_count, total=   0.4s
[Pipeline] ... (step 3 of 37) Processing rating_3_count, total=   0.1s
[Pipeline] ... (step 4 of 37) Processing rating_4_count, total=   0.2s
[Pipeline] ... (step 5 of 37) Processing rating_5_count, total=   0.1s
[Pipeline]  (step 6 of 37) Processing comments_published_count, total=   0.1s
[Pipeline]  (step 7 of 37) Processing photos_published_count, total=   0.1s
[Pipeline]  (step 8 of 37) Processing videos_published_count, total=   0.1s
[Pipeline] .. (step 9 of 37) Processing PriceDiscounted, total=   0.1s
[Pipeline] . (step 10 of 37) Processing item_time_alive, total=   0.1s
[Pipeline]  (step 11 of 37) Processing item_count_fake_returns7, total=   0.1s
[Pipeline]  (step 12 of 37) Processing item_count_fake_returns30, total=   0.1s
[Pipeline]  (step 13 of 37) Processing item_count_fake_returns90, total=   0.2s
[Pipeline]  (step 14 of 37) Proces

In [ ]:
transformed_train = num_prep_pipe.transform(train, is_train=True)
transformed_test = num_prep_pipe.transform(test, is_train=False)

/home/den/miniconda3/envs/ozon-e-cup/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/den/miniconda3/envs/ozon-e-cup/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [ ]:
train[train["PriceDiscounted"] < 0].shape[0], train.PriceDiscounted.isna().sum(), transformed_train[
    transformed_train["PriceDiscounted"] == -1
].shape[0] / train.shape[0], transformed_train[transformed_train["PriceDiscounted"] == -1].shape[0]

(0, np.int64(0), 0.0005730281240174849, 113)

In [ ]:
test[test["PriceDiscounted"] < 0].shape[0], test.PriceDiscounted.isna().sum(), transformed_test[
    transformed_test["PriceDiscounted"] == -1
].shape[0] / test.shape[0], transformed_test[transformed_test["PriceDiscounted"] == -1].shape[0]

(0, np.int64(3), 0.00013181019332161688, 3)

# Categorical features preprocessing

In [ ]:
# Переведем SellerID в строку (надо для корректной работы Categorical трансформа)
train["SellerID"] = train["SellerID"].astype(str)
test["SellerID"] = test["SellerID"].astype(str)

In [ ]:
cat_prep_pipe = prep.CATEGORIAL_PREP_PIPELINE
cat_prep_pipe = cat_prep_pipe.fit(train)

[Pipeline] ........ (step 1 of 3) Processing brand_name, total=   0.5s
[Pipeline]  (step 2 of 3) Processing CommercialTypeName4, total=   0.5s
[Pipeline] .......... (step 3 of 3) Processing SellerID, total=   0.2s


In [ ]:
transformed_train = cat_prep_pipe.transform(train)
transformed_train.head(3)

,id,resolution,description,name_rus,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,comments_published_count,...,SellerID_4,SellerID_5,SellerID_6,SellerID_7,SellerID_8,SellerID_9,SellerID_10,SellerID_11,SellerID_12,SellerID_13
0,159385,0,"Мешки пылесборники для пылесоса PHILIPS, 10 шт...","Мешки для пылесоса PHILIPS TRIATLON, синтетиче...",6.0,4.0,4.0,3.0,32.0,3.0,...,0,0,0,0,0,0,0,0,0,1
1,288616,0,Защитная силиконовая крышка обьектива GoPro He...,Защитная крышка Redline на экшн-камеру GoPro (...,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
2,108090,0,Плоский медиатор из кости толщиной 0.6 мм<br/>...,Медиатор для гитары Acura GP-PB6,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,1,1
